In [60]:
# 2022年スマブラー格付けチェックの回答をTwitterAPIで取得し、簡単な加工をしてcsv出力を行う

In [ ]:
# 収集した回答の96%が正解のCの部屋であり、分析困難なのでこちらのコードだけリファクタリングした

### 初期設定

In [1]:
import tweepy
import datetime
from pytz import timezone
from collections import defaultdict
import pandas as pd
import glob

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option("display.max_colwidth", 80)

### API準備

In [5]:
# 申請･取得したキーやトークンを入力する
# プログラム公開時はマスキングすること！！！

API_KEY = 'xxxxxxxx'
API_SECRET_KEY = 'xxxxxxxx'
Access_token = 'xxxxxxxx'
Access_secret = 'xxxxxxxx'

In [6]:
# API分析用の変数格納

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(Access_token, Access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) # wait_on_rate_limit=True, API上限到達時に自動で待機する

### 関数定義

In [ ]:
def get_now():
    """
    yyyyMMddHHmmss形式で現在の日本時間を出力
    """
    now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))) # 日本時刻
    return now.strftime('%Y%m%d%H%M%S')

### データ取得

In [ ]:
# ｢Aの部屋に投票した人｣と判定する条件 (B,C の部屋についても同様に定める)
# 1. 出題ツイート投稿(2022/12/29 18:03)から 解答ツイート投稿の直前(2022-12-30_17:59:59_JST)の間にツイートした
# 2. Aの部屋 のハッシュタグを含み #Bの部屋, #Cの部屋 のハッシュタグを両方含まないツイートをした

In [50]:
# 出題ツイートID
nietono_q_tw_id = 1608388104695738370

In [51]:
# 解答ツイートID
nietono_ans_tw_id = 1608749942423826432

In [58]:
# 解答ツイート投稿の直前の時刻を設定
# tweepy.CursorでsinceはID指定できるがuntilはID指定できない

nietono_ans_tw_time = (api
                       .get_status(nietono_ans_tw_id)
                       .created_at
                       .astimezone(timezone('Asia/Tokyo'))
                       - datetime.timedelta(seconds=1) # 投稿1秒前
                      ).strftime('%Y-%m-%d_%H:%M:%S_JST')
nietono_ans_tw_time

'2022-12-30_18:00:49_JST'

In [59]:
# 回答ツイートとユーザーの情報を取得

room_names = ["A", "B", "C"]
q_dict = {"A":"#Aの部屋 AND -#Bの部屋 AND -#Cの部屋",
          "B":"#Bの部屋 AND -#Cの部屋 AND -#Aの部屋",
          "C":"#Cの部屋 AND -#Aの部屋 AND -#Bの部屋"
         }
tw_data = defaultdict(list)

for vote_to in room_names:
    for tweet in tweepy.Cursor(api.search_tweets, q=q_dict[vote_to], since_id=nietono_q_tw_id, until=nietono_ans_tw_time, result_type="mixed").items():
        tw_data["vote"].append(vote_to)
        tw_data["tw_time"].append(tweet.created_at.astimezone(timezone('Asia/Tokyo'))) # JSTに修正
        tw_data["tw_text"].append(tweet.text)
        tw_data["twitter_user_id"].append(tweet.user.id)
        tw_data["twitter_user_name"].append(tweet.user.name)
        tw_data["twitter_user_prof"].append(tweet.user.description)
        tw_data["twitter_user_screen_name"].append(tweet.user.screen_name)
        tw_data["twitter_user_followers_count"].append(tweet.user.followers_count)
        tw_data["vote_url"].append(f"twitter.com/{tweet.user.screen_name}/status/{tweet.id}")
        tw_data["is_quote"].append(tweet.is_quote_status)

df_vote = pd.DataFrame.from_dict(tw_data)
df_vote.to_csv("df_vote/df_vote_" + get_now() + ".csv", index=False, encoding="utf-8-sig") # 時刻付きで出力
df_vote

,vote,tw_time,tw_text,twitter_user_id,twitter_user_name,twitter_user_prof,twitter_user_screen_name,twitter_user_followers_count,vote_url,is_quote
0,A,2022-12-30 16:36:05+09:00,「やっぱ、そうだと思ってたんよ。」\nって言う準備だけは万端整っております。\n#スマブラー格付けチェック\n#Aの部屋 https://t.co/nA...,1267297184698920961,猫山田部長🐾,猫山田部長です。\n名前の通り、世の中の役に立たない部の部長をやっとります。\n音速のいいねには定評があります。４歳サバトラです。,NekoyamadaBucho,1382,twitter.com/NekoyamadaBucho/status/1608728616053116929,False
1,A,2022-12-30 14:54:47+09:00,RT @Ueji1123: 結果はっぴょ～う！\n答えは #Aの部屋 です！\nAの部屋は しゅーとん vs KEN、Bの部屋は つぼつぼ vs にぐと...,1284505913152303111,しーくん\ビシルプ,20歳のガキです||スプラトゥーンを愛してるスマブラ勢||思ったことなんでも呟くからたまにうるさい||フィットレメイン、イカカズヤ使いたい||ゼノブレフ...,shiikun_SSB,113,twitter.com/shiikun_SSB/status/1608703123035672580,False
2,A,2022-12-30 14:18:58+09:00,RT @Ueji1123: 結果はっぴょ～う！\n答えは #Aの部屋 です！\nAの部屋は しゅーとん vs KEN、Bの部屋は つぼつぼ vs にぐと...,1511997531043303425,なんとか,現実と向き合うアカウント \nコントラクトブリッジ スマブラSP\n現実から逃げるアカウント→@somethin_tonika,ludossbu,26,twitter.com/ludossbu/status/1608694110927589376,False
3,A,2022-12-30 13:52:42+09:00,RT @chibiT2169: #スマブラー格付けチェック \nこれは間違いなく #Aの部屋 です。復帰台から降りてきた時のピクミンの引っこ抜き方で判断...,1188408449127698433,RaiNa Chronicle,メイン ディディー サブ ウルフ パルテナ 全ての原点はトレモだと思ってる。,RaiNa53581894,401,twitter.com/RaiNa53581894/status/1608687498720587777,False
4,A,2022-12-30 13:09:58+09:00,RT @chibiT2169: #スマブラー格付けチェック \nこれは間違いなく #Aの部屋 です。復帰台から降りてきた時のピクミンの引っこ抜き方で判断...,954442172207525888,コカ・コウラ,沢山尊い絵とエッチな絵をRTしてる人,cola_yummy_,499,twitter.com/cola_yummy_/status/1608676746907521025,False
...,...,...,...,...,...,...,...,...,...,...
925,C,2022-12-29 18:18:07+09:00,#スマブラー格付けチェック #Cの部屋,979270253585711105,はるる,お別れはブロ解で たまに荒れるのでフォローは自己責任で\n質問箱:https://t.co/zmoqwShj6P気軽にどうぞ,haruru__hero,617,twitter.com/haruru__hero/status/1608391906349973504,False
926,C,2022-12-29 18:17:57+09:00,#スマブラー格付けチェック #Cの部屋,1414227889143177216,デルガド,趣味垢・ぱせリス 友達が欲しい,ddddrrrgddddd,28,twitter.com/ddddrrrgddddd/status/1608391864910249985,False
927,C,2022-12-29 18:17:10+09:00,#スマブラー格付けチェック #Cの部屋,1137328153431437312,あisu🍨\nスティーブ消せ,春日野穹 Love,Aisu_Sheik,800,twitter.com/Aisu_Sheik/status/1608391667081678848,False
928,C,2022-12-29 18:14:19+09:00,流石に今回露骨じゃない？余裕のCです\n#スマブラー格付けチェック #Cの部屋,714790604081799168,PlusQ,北大地惑B4､スマサー SSBU:Steve Wolf ブルアカ ASMR狂信者 いいねをタイムラインに表示している君を社会的に抹殺する可能性のあるアカウント,93ta_im,129,twitter.com/93ta_im/status/1608390948773593088,False


### データ加工･出力

In [33]:
# 何度か分けて取得したデータをマージ。各ユーザーごとに最後の回答だけ残す
# 回答修正と、ツイート削除対策

df_vote = pd.DataFrame()

for file in glob.glob("df_vote/*"):
    df_vote = pd.concat([df_vote, pd.read_csv(file)])

df_vote = (df_vote
           .sort_values(["tw_time"])
           .drop_duplicates("user_id", keep="last")
          )

In [34]:
# 回答ツイートのRTを除く
# 単純RTや、引用RTのRT(例:1476301813599068161)はtweet.text[0:4]=="RT @"となることを利用する

df_vote = df_vote[df_vote.tw_text.str[0:4] != "RT @"]

In [37]:
# 出題後何時間後のツイートかカラム追加

df_vote["delta_hours"] = ((df_vote["tw_time"]
                          .apply(pd.to_datetime)-pd.to_datetime("2021-12-29 18:00:00+09:00"))
                          .dt.total_seconds()
                          .apply(lambda x: int(x/3600))
                         )

In [62]:
# 出力

df_vote.to_csv("df_vote.csv", index=False, encoding="utf-8-sig")
df_vote